In [1]:
import sys

In [2]:
!{sys.executable} -m pip install crdch_model linkml black

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [36]:
import crdch_model.crdch_model as ccdh
from linkml_runtime.utils.schemaview import SchemaView
from linkml_runtime.dumpers import yaml_dumper
import requests
import pandas as pd

In [4]:
crdch_linkml_file = "../ccdhmodel/model/schema/crdch_model.yaml"

In [5]:
crdch_view = SchemaView(crdch_linkml_file)

In [6]:
# crdch_schema = crdch_view.schema

----

In [7]:
cso1 = ccdh.CancerStageObservation(
    observation_type=ccdh.CodeableConcept(),
    value_codeable_concept=ccdh.CodeableConcept(),
)

In [8]:
csos = ccdh.CancerStageObservationSet()

In [9]:
csos.observations.append(cso1)

In [10]:
print(csos)

CancerStageObservationSet(id=None, category=None, focus=[], subject=None, method_type=[], performed_by=None, observations=[CancerStageObservation(observation_type=CodeableConcept(coding=[], text=None), value_codeable_concept=CodeableConcept(coding=[], text=None), id=None, category=None, method_type=None, focus=None, subject=None, performed_by=None, value_entity=None)])


## Time out
- `CancerStageObservation` requires `CodeableConcept`s for `observation_type` and `value_codeable_concept`, but those `CodeableConcepts` can be empty
- `id` can be empty!
- `CancerStageObservation` isn't asserted to be a subclass of `Observation`, nor `CancerStageObservationSet` as `ObservationSet`

_show by opening the model into a schemaview_

In [11]:
CancerStageObservation_view = crdch_view.get_class("CancerStageObservation")
print(CancerStageObservation_view.is_a)

Entity


In [12]:
print(yaml_dumper.dumps(CancerStageObservation_view))

name: CancerStageObservation
description: A data structure with key (observation_type) and value (value_codeable_concept)
  attributes that represents a single cancer staging observation, such as the Clinical
  Metastasis (M) component of a clinical TNM staging.
notes:
- Derived from [CancerStageObservation in sheet CancerStageObservation](https://docs.google.com/spreadsheets/d/1oWS7cao-fgz2MKWtyr8h2dEL9unX__0bJrWKv6mQmM4/edit#gid=1189722404)
is_a: Entity
attributes:
  id:
    name: id
    description: The 'logical' identifier of the entity within the system of record.  The
      simple value of this attribute stands for an identifier of this data object
      within the system, it can be used as a reference from other objects within the
      same system (i.e. primary and foreign keys), and it should be unique per type
      of object. The same data object copied to a different system will likely have
      a different "id" in the new system since "id" values are system specific and
 

In [13]:
dgcc = ccdh.CodeableConcept(text="pizza doesn't taste good anymore")

In [14]:
dgc = ccdh.Coding(
    code="any text acceptable", system="not even a URL", label="dysgeusia"
)

In [15]:
dgc

Coding(code='any text acceptable', system='not even a URL', label='dysgeusia', system_version=None, value_set=None, value_set_version=None, tag=[])

In [16]:
dgcc.coding.append(dgc)

In [17]:
dgcc

CodeableConcept(coding=[Coding(code='any text acceptable', system='not even a URL', label='dysgeusia', system_version=None, value_set=None, value_set_version=None, tag=[])], text="pizza doesn't taste good anymore")

## TIme out

- if a Coding is assigned to a `CodeableConcept`, it will require a `code` and a `system`, but those can be plain strings
- `Coding`s should be **appended** to `CodeableConcept`s.
- `CodeableConcept` doesn have subtypes, like CancerStageObservationTypeCodeableConcept


In [18]:
crdch_view.class_descendants("CodeableConcept")

['CodeableConcept']

In [19]:
cso2 = ccdh.CancerStageObservation(observation_type=dgcc, value_codeable_concept=dgcc)

In [20]:
# start over
csos2 = ccdh.CancerStageObservationSet()
csos2.observations.append(cso2)

In [21]:
csos2

CancerStageObservationSet(id=None, category=None, focus=[], subject=None, method_type=[], performed_by=None, observations=[CancerStageObservation(observation_type=CodeableConcept(coding=[Coding(code='any text acceptable', system='not even a URL', label='dysgeusia', system_version=None, value_set=None, value_set_version=None, tag=[])], text="pizza doesn't taste good anymore"), value_codeable_concept=CodeableConcept(coding=[Coding(code='any text acceptable', system='not even a URL', label='dysgeusia', system_version=None, value_set=None, value_set_version=None, tag=[])], text="pizza doesn't taste good anymore"), id=None, category=None, method_type=None, focus=None, subject=None, performed_by=None, value_entity=None)])

In [22]:
csos_yaml = yaml_dumper.dumps(csos)
print(csos_yaml)

{}



In [23]:
csos2_yaml = yaml_dumper.dumps(csos2)
print(csos2_yaml)

observations:
- observation_type:
    coding:
    - code: any text acceptable
      system: not even a URL
      label: dysgeusia
    text: pizza doesn't taste good anymore
  value_codeable_concept:
    coding:
    - code: any text acceptable
      system: not even a URL
      label: dysgeusia
    text: pizza doesn't taste good anymore



## Round trip:
Build a data object, convert to YAML (string or file) and then use that to recreate a new data object

In [24]:
from linkml_runtime.loaders import yaml_loader



In [25]:
yaml_loader.load(csos_yaml, ccdh.CancerGradeObservationSet)

CancerGradeObservationSet(id=None, category=None, focus=[], subject=None, method_type=[], performed_by=None, observations=[])

In [26]:
yaml_loader.load(csos2_yaml, ccdh.CancerGradeObservationSet)

TypeError: unhashable type: 'CodeableConcept'

----

- where can we learn about mappings like XXX?
    - mappings within model
    - terminology service

In [42]:
def get_mapping_table(model_name, class_name, slot_name):
    global global_mapping_table
    current_url = f"https://terminology.ccdh.io/models/{model_name}/entities/{class_name}/attributes/{slot_name}/mappings"
    response = requests.get(current_url)
    resp_struct = response.json()
    current_mappings = pd.DataFrame(resp_struct["mappings"])


In [43]:
mapping_res = get_mapping_table("CRDC-H","BodySite","site")

In [44]:
print(mapping_res)

None
